# Process the data (combine them together)

## Check JH's augmentation data

In [ ]:
import importlib
import Functions.MyDataset as MyDataset
import Functions.Visualization as Visualization

JH_aug_mat_path = "/Volumes/Shawn_SSD/PhD/Project/Date/Tmp/JH_Pre_Augmentation/original_augmentation_data/JH_aug_2.mat"
JH_reshape_vol_mat_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Val/Input/FA_17017030_AugVol_2.mat"
JH_reshape_pts_mat_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Val/Output/FA_17017030_AugPts_2.mat"

pixel_space = [0.15, 0.15, 0.15]

importlib.reload(MyDataset)

# JH_aug_volume, JH_aug_pts = functions.load_mat_data(JH_aug_mat_path)
# functions.show_pts(JH_aug_volume, JH_aug_pts, pixel_space)

JH_aug_zoomed_volume, JH_aug_zoomed_pts = MyDataset.load_mat_data(JH_reshape_vol_mat_path, JH_reshape_pts_mat_path)
Visualization.show_pts(JH_aug_zoomed_volume, JH_aug_zoomed_pts, pixel_space)


In [ ]:
# Try up_sampling method

up_sampling_JH = MyDataset.rescale_3d_volume(JH_aug_zoomed_volume, (512, 512, 90))
Visualization.show_pts(up_sampling_JH, JH_aug_zoomed_pts, pixel_space)

## Check rescaled data

In [ ]:
import importlib
import Functions

# data_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_rescaled_data/176_176_48/AH_17617648_VolPts_3.mat"
# data_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_rescaled_data/240_240_64/AH_24024064_VolPts_3.mat"
# data_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_rescaled_data/320_320_96/AH_32032096_VolPts_3.mat"
# data_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_rescaled_data/480_480_128/AH_480480128_VolPts_3.mat"
data_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_rescaled_data/560_560_160/AH_560560160_VolPts_3.mat"

aug_rescaled_volume, aug_rescaled_pts = MyDataset.load_mat_data(data_path)

pixel_space = [0.15, 0.15, 0.15]
Visualization.show_pts(aug_rescaled_volume, aug_rescaled_pts, pixel_space)

## Combination

In [ ]:
import numpy as np
import importlib
import Functions

def combine_data(x_base_path, y_base_path, store_base_path, data_tag):
    importlib.reload(MyDataset)

    # X_val_base_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Val/Input"
    # Y_val_base_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Val/Output"

    x_dataset, y_dataset = MyDataset.load_mat_data_dir(x_base_path, y_base_path)

    x_dataset = np.asarray(x_dataset)
    y_dataset = np.asarray(y_dataset)

    x_dataset_path = store_base_path + "X_" + data_tag + "_data"
    y_dataset_path = store_base_path + "Y_" + data_tag + "_data"

    np.save(x_dataset_path, x_dataset)
    np.save(y_dataset_path, y_dataset)


In [ ]:
X_train_base_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Train/Input"
Y_train_base_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Train/Output"
X_val_base_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Val/Input"
Y_val_base_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Val/Output"
X_test_base_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Test/Input"
Y_test_base_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Test/Output"
store_base_path = "data/"

combine_data(X_train_base_path, Y_train_base_path, store_base_path, "train")
combine_data(X_val_base_path, Y_val_base_path, store_base_path, "val")
combine_data(X_test_base_path, Y_test_base_path, store_base_path, "test")

In [ ]:
import numpy as np
import Functions

# Inspect Loaded Data

X_file_path = "data/X_train_data.npy"
Y_file_path = "data/Y_train_data.npy"

X_load = np.load(X_file_path, allow_pickle=True)
Y_load = np.load(Y_file_path, allow_pickle=True)

print(np.shape(X_load), np.shape(Y_load))
print(np.shape(X_load[1, :, :, :]), np.shape(Y_load[1, :, :]))

# functions.show_pts(X_load[1, :, :, :], Y_load[1, :, :], [0.15, 0.15, 0.15])
# functions.show_pts(X_load[2, :, :, :], Y_load[2, :, :], [0.15, 0.15, 0.15])
# functions.show_pts(X_load[3, :, :, :], Y_load[3, :, :], [0.15, 0.15, 0.15])
# functions.show_pts(X_load[4, :, :, :], Y_load[4, :, :], [0.15, 0.15, 0.15])

## Check slr model

In [29]:
import Spartan.models as models
import importlib

importlib.reload(models)

model = models.spine_lateral_radiograph_model(width=32, height=32, depth=16)
model.summary()

Model: "ResModel"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 32, 32, 16,  0           []                               
                                 1)]                                                              
                                                                                                  
 batch_normalization_150 (Batch  (None, 32, 32, 16,   4          ['input_9[0][0]']                
 Normalization)                 1)                                                                
                                                                                                  
 re_lu_150 (ReLU)               (None, 32, 32, 16,   0           ['batch_normalization_150[0][0]']
                                1)                                                         

In [24]:
import numpy as np

importlib.reload(models)

# check the pre-defined coordinate matrix
batch_base_cor_xyz = models.coordinate_3d(2,2,3,4)

# test base coordinate matrix
print("batch_base_cor_xyz shape: ", np.shape(batch_base_cor_xyz))
print(batch_base_cor_xyz[0, :, :, 1, 0, 1])

batch_base_cor_xyz shape:  (2, 2, 3, 4, 4, 3)
[[-0.5 -0.5 -0.5]
 [ 0.5  0.5  0.5]]


In [ ]:
from tensorflow import keras

# draw a graph for the model's
# keras.utils.plot_model(model_s1, "my_first_model.png")

In [25]:
import numpy as np

# check output of the model
input = np.arange(2*32*32*16*1).reshape(2,32,32,16,1)
base_cor_xyz = models.coordinate_3d(2,32,32,16)

[output_s1, output_s2] = model([input, base_cor_xyz])

print(np.shape(output_s1))
print(np.shape(output_s2))


(2, 4, 3)
(2, 4, 3)
tf.Tensor(
[[[ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
  [-3.7081023e-31  3.2296310e-31  4.0625000e-01]]

 [[-1.4500000e+01  1.3250000e+01  0.0000000e+00]
  [ 1.2812500e+01  1.0125000e+01  0.0000000e+00]
  [-1.3500000e+01  1.1687500e+01  0.0000000e+00]
  [-1.4531250e+01 -1.1843750e+01 -4.0625000e-01]]], shape=(2, 4, 3), dtype=float32)
tf.Tensor(
[[[ 1.3566972e+00 -1.0316093e+00  3.4818423e-01]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
  [-9.0167046e-01 -9.6806896e-01  9.3714631e-01]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]]

 [[-2.6517220e+00  7.6928005e+00 -3.4818298e-01]
  [-9.0614929e+00 -1.7180436e+00 -3.5762787e-06]
  [ 4.6875000e-01 -1.4406254e+01 -9.3716455e-01]
  [ 2.2050784e+00  1.4500172e+01  3.9279461e-05]]], shape=(2, 4, 3), dtype=float32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerT

In [26]:
importlib.reload(models)

# check loss function
y_true = np.asarray([[[1,2,3],[2,3,4],[6,7,8],[7,8,9]], [[1.5,2.5,3.5],[2.5,3.5,4.5],[6.5,7.5,8.5],[7.5,8.5,9.5]]])

y_pred1 = np.asarray([[[1,2,3],[2,3,4],[6,7,8],[7,8,9]], [[1.5,2.5,3.5],[2.5,3.5,4.5],[6.5,7.5,8.5],[7.5,8.5,9.5]]])
y_pred2 = np.asarray([[[1,2,3],[2,3,4],[6,7,8],[7,8,9]], [[1.5,2.5,3.5],[2.5,3.5,4.5],[6.5,7.5,8.5],[7.5,8.5,9.5]]])

loss = models.two_stage_wing_loss(y_true, [output_s1, output_s2])

print(loss)

tf.Tensor(347.10425, shape=(), dtype=float32)
